[Google colab version](https://colab.research.google.com/drive/15wJus5WZPYxTYE51yBhIBNhk9Tj4k3BT?usp=sharing)

# 3D segmentation with [MONAI](https://github.com/Project-MONAI/MONAI) and [Catalyst](https://github.com/catalyst-team/catalyst)

This tutorial demonstrates how [MONAI](https://github.com/Project-MONAI/MONAI) can be used with the [Catalyst](https://github.com/catalyst-team/catalyst) framework for 3D segmentation task.
And easily use below features:

- Prepare synthetic data.
- Load Nifti image with metadata.
- Transforms for dictionary format data.
- Add channel dim to the data if no channel dimension.
- Scale medical image intensity with expected range.
- Crop out a batch of balanced images based on positive / negative label ratio.
- 3D UNet model, Dice loss function, Mean Dice metric for 3D segmentation task.
- Sliding window inference method.
- Deterministic training for reproducibility.

This tutorial is based on [unet_training_dict.py](https://github.com/Project-MONAI/MONAI/blob/master/examples/segmentation_3d/unet_training_dict.py) and [spleen_segmentation_3d.ipynb](https://github.com/Project-MONAI/MONAI/blob/master/examples/notebooks/spleen_segmentation_3d.ipynb).

## Install requirements

In [ ]:
! pip install -q catalyst==20.07

In [ ]:
import os
import sys
import tempfile
import shutil
from glob import glob
import logging
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.transforms import (
    Compose,
    LoadNiftid,
    AsChannelFirstd,
    ScaleIntensityd,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ToTensord,
)
from monai.data import create_test_image_3d, list_data_collate
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.visualize import plot_2d_or_3d_image

from catalyst import dl

monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# [MONAI](https://github.com/Project-MONAI/MONAI) components

## Prepare synthetic data

In [ ]:
# create a temporary directory and 40 random image, mask paris
tempdir = tempfile.mkdtemp()
print(f"generating synthetic data to {tempdir} (this may take a while)")
for i in range(40):
    im, seg = create_test_image_3d(128, 128, 128, num_seg_classes=1, channel_dim=-1)

    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(tempdir, f"img{i:d}.nii.gz"))

    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(tempdir, f"seg{i:d}.nii.gz"))

images = sorted(glob(os.path.join(tempdir, "img*.nii.gz")))
segs = sorted(glob(os.path.join(tempdir, "seg*.nii.gz")))

## Prepare transforms and datasets

In [ ]:
train_files = [{"img": img, "seg": seg} for img, seg in zip(images[:20], segs[:20])]
val_files = [{"img": img, "seg": seg} for img, seg in zip(images[-20:], segs[-20:])]

# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadNiftid(keys=["img", "seg"]),
        AsChannelFirstd(keys=["img", "seg"], channel_dim=-1),
        ScaleIntensityd(keys=["img", "seg"]),
        RandCropByPosNegLabeld(
            keys=["img", "seg"], label_key="seg", spatial_size=[96, 96, 96], pos=1, neg=1, num_samples=4
        ),
        RandRotate90d(keys=["img", "seg"], prob=0.5, spatial_axes=[0, 2]),
        ToTensord(keys=["img", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadNiftid(keys=["img", "seg"]),
        AsChannelFirstd(keys=["img", "seg"], channel_dim=-1),
        ScaleIntensityd(keys=["img", "seg"]),
        ToTensord(keys=["img", "seg"]),
    ]
)

# define dataset, data loader
check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
check_loader = DataLoader(check_ds, batch_size=2, num_workers=4, collate_fn=list_data_collate)
check_data = monai.utils.misc.first(check_loader)
print(check_data["img"].shape, check_data["seg"].shape)

# create a training data loader
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)
# create a validation data loader
val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=4, collate_fn=list_data_collate)

## Prepare model, optimizer and metrics

In [ ]:
# create UNet, DiceLoss and Adam optimizer
# device = torch.device("cuda:0")  # you don't need device, because Catalyst uses autoscaling
model = monai.networks.nets.UNet(
    dimensions=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
loss_function = monai.losses.DiceLoss(sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

dice_metric = DiceMetric(include_background=True, to_onehot_y=False, sigmoid=True, reduction="mean")

# [Catalyst](https://github.com/catalyst-team/catalyst) experiment

## Setup Runner

In [ ]:
class MonaiSupervisedRunner(dl.SupervisedRunner):

  def forward(self, batch):
    if self.is_train_loader:
      output = {self.output_key: self.model(batch[self.input_key])}
    elif self.is_valid_loader:
      roi_size = (96, 96, 96)
      sw_batch_size = 4
      output = {self.output_key: 
                sliding_window_inference(batch[self.input_key], roi_size, sw_batch_size, self.model)}
    elif self.is_infer_loader:
      roi_size = (96, 96, 96)
      sw_batch_size = 4
      batch = self._batch2device(batch, self.device)
      output = {self.output_key: 
                sliding_window_inference(batch[self.input_key], roi_size, sw_batch_size, self.model)}
      output = {**output, **batch}
    return output

## Run experiment

In [ ]:
runner = MonaiSupervisedRunner(
    input_key="img", input_target_key="seg", output_key="logits")  # you can also specify `device` here
runner.train(
    loaders={"train": train_loader, "valid": val_loader},
    model=model,
    criterion=loss_function,
    optimizer=optimizer,
    num_epochs=6, logdir="./logs", 
    main_metric="dice_metric", minimize_metric=False,
    verbose=False, timeit=True,  # let's use minimal logs, but with time checkers
    callbacks={
        "loss": dl.CriterionCallback(input_key="seg", output_key="logits"),
        "periodic_valid": dl.PeriodicLoaderCallback(valid=2),
        "dice_metric": dl.MetricCallback(prefix="dice_metric", metric_fn=dice_metric, 
                                         input_key="seg", output_key="logits")
    },
    load_best_on_end=True,  # user-friendly API :)
)

# Tensorboard logs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir='logs'

# Best model performance visualisation

In [ ]:
import matplotlib.pyplot as plt

for i, valid_output in enumerate(runner.predict_loader(loader=val_loader)):
    if i > 4:
        break
    plt.figure('check', (9, 3))
    plt.subplot(1, 3, 1)
    plt.title('image ' + str(i))
    plt.imshow(valid_output['img'].detach().cpu()[0, 0, :, :, 48], cmap='gray')
    plt.subplot(1, 3, 2)
    plt.title('label ' + str(i))
    plt.imshow(valid_output['seg'].detach().cpu()[0, 0, :, :, 48])
    plt.subplot(1, 3, 3)
    plt.title('output ' + str(i))
    logits = valid_output["logits"]
    plt.imshow((logits[0] > 0.5).float().detach().cpu()[0, :, :, 48])
    plt.show()

In [ ]:
shutil.rmtree(tempdir)